First, we need to do our imports.

In [298]:
import numpy as np
import pandas as pd

Now we need to import the data:

In [299]:
training_data = pd.read_csv('./train.csv')
data = np.array(training_data)
m, n = data.shape

# Shuffle the data to ensure a good mix 
np.random.shuffle(data) 

# Split the data into development (validation) and training sets
data_dev = data[0:1000].T  # Test set
Y_dev = data_dev[0]        
X_dev = data_dev[1:]       
X_dev = X_dev / 255.       

data_train = data[1000:].T  #Training set 
Y_train = data_train[0]     
X_train = data_train[1:]    
X_train = X_train / 255. 


So we have 37800 training images, per image we have 28*28=784 pixels, each pixel has a number from 0 to 255 representing it's color on the greyscale.
Now we need to make a function that initializes the parameters:

In [300]:
def init_parameters():
    #First layer's parameters
    #This makes an array of 10 arrays, each array has 784 elements, that are random between -0.5 and 0.5
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5

    # #Second layer's parameters
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1,b1,W2,b2


Now we start with forward propagation:
But before, we need to define an activation function this can be Sigmoid, ReLU or whatever you choose, for this example, we can choose ReLU:

In [301]:
def ReLU(array):
    #This does it element wise, which means for every element if it's positive it will return the element
    #If negative, it returns 0.
    return np.maximum(0,array)

We also need a softmax function:

In [302]:
def softmax(array):
    #This is just a mathematical formula, it adds up all the sums, then say the sum of the array is 
    #20, and the value for three is 7, then three will have 7/20, which is the probability that this number 
    #is indeed a 3
    return np.exp(array)/sum(np.exp(array))

In [303]:
def forward_propagation(W1,b1,W2,B2,input):
    #First layer here refers to the first hidden layer, because the input layer doesn't count
    Z1= W1.dot(input)+b1
    A1=ReLU(Z1)
    #Now for the second layer, we want to get a probability, i.e a number between 0 and 1, which means
    #we need to do a softmax function
    Z2=W2.dot(A1)+B2
    A2=softmax(Z2)
    return Z1,A1,Z2,A2
    

Now for back propagation, which is basically the step that uses gradient calculus, to find the lowest error, which will be the minima of the cost function, every time we run a training example, we do forward propagation, it will give an assumption that is usually not great at the beggining, then we will tell the computer just how off it is from the actual values that it should have in the last layer, the last layer will tell the layer before it how the weights and biases should be changed, and that layer's values will tell the layer before it how the weights and biases should be changed and so on.

In [304]:
#So we basically give this function an array of numbers, for each number, it will give 
#an array of 10 elements, where only the index of that number is 1, and the rest is zero
#we do this so we can subtract the predicted value from the true value, and for each example
#get an array of the difference. 
def one_hot(Y,num_classes=None):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

In [306]:
def ReLU_deriv(Z):
    return Z > 0
def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    #To keep track the difference will look something like this:
    #difference = [0.3-0, 0.03-0, 0.8-1, ....]

    dW2 = 1 / m * dZ2.dot(A1.T) #Adding the transpose bascially means division
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    
    #Repeat for first layer:
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    #So these derivates now show us the gradient step, which is in the steepest ascent direction
    #What we want to do is go in the opposite direction (in the negative direction)
    #So we can find the steepest descent
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [307]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

#Putting everything together
def gradient_descent(X, Y, alpha, iterations):
    #Initialize the Parameters
    W1, b1, W2, b2 = init_parameters()

    #Now we will start the learning loop:
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_propagation(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 35 == 0:
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [308]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.1, 500)

[9 5 6 ... 6 9 6] [7 0 0 ... 7 9 2]
0.09760975609756098
[3 0 0 ... 9 9 2] [7 0 0 ... 7 9 2]
0.3903170731707317
[3 0 0 ... 9 9 2] [7 0 0 ... 7 9 2]
0.514609756097561
[3 0 0 ... 7 9 2] [7 0 0 ... 7 9 2]
0.630390243902439
[3 5 0 ... 7 9 2] [7 0 0 ... 7 9 2]
0.7007073170731707
[3 5 0 ... 7 9 2] [7 0 0 ... 7 9 2]
0.7432682926829268
[3 5 0 ... 7 9 2] [7 0 0 ... 7 9 2]
0.7721463414634147
[3 5 0 ... 7 9 2] [7 0 0 ... 7 9 2]
0.7940975609756098
[3 5 0 ... 7 9 2] [7 0 0 ... 7 9 2]
0.8101463414634147
[2 5 0 ... 7 9 2] [7 0 0 ... 7 9 2]
0.820780487804878
[2 5 0 ... 7 9 2] [7 0 0 ... 7 9 2]
0.8290975609756097
[2 5 0 ... 7 9 2] [7 0 0 ... 7 9 2]
0.8354390243902439
[2 5 0 ... 7 9 2] [7 0 0 ... 7 9 2]
0.8414390243902439
[2 5 0 ... 7 9 2] [7 0 0 ... 7 9 2]
0.8465853658536585
[2 5 0 ... 7 9 2] [7 0 0 ... 7 9 2]
0.850780487804878


We get up to 85% accuracy on the training set.